In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive/')
os.chdir("/content/gdrive/MyDrive/DISSERTATION")
print(os.listdir())

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
['simple_hilbert.py', '__pycache__', 'Flowpast', 'MeshCNN', 'cache', 'fpc_000.npz (Unzipped Files)', 'Advection_of_a_block_over_structured_mesh.ipynb', 'fpc2.obj', 'Diss.ipynb', 'csv_data', 'fpc3.obj', 'TENS', 'Diss2.ipynb', 'AE with FC.ipynb', 'Diss3.ipynb', 'Compressed.obj', 'AverageAndNormalise.ipynb', 'CAE_on_structured_mesh.ipynb', 'PyTorch Geometric attempt 2', 'MoreChannels.ipynb', 'animation', 'original_simulation.mp4', 'Compress.ipynb', 'Copy of MoreChannels.ipynb', 'OLDMULTIPLE', 'MultipleCurves', 'Cleanedd.obj', 'Cleaned2.obj', 'Cleaned3.obj', 'Cleaned4.obj', 'Cleaned5.obj', 'Cleaned6.obj', 'figure.png', 'Cleaned7.obj', 'Cleaned8.obj', 'Cleaned9.obj', 'popped.obj', 'lessee.obj', 'Compress2.ipynb', 'GCAE.ipynb', 'Presentation.gslides']


In [2]:
#Importing a bunch of stuff from MeshCNN
os.chdir("./MeshCNN")
print(os.listdir())
import models
import options.train_options
os.chdir("./util")
import mesh_viewer
os.chdir("..")
os.chdir("./models")
import networks
os.chdir("./layers")
import mesh
import mesh_pool
import mesh_unpool
import mesh_union
os.chdir("../../../")

['.git', '.gitignore', '.travis.yml', 'LICENSE', 'README.md', 'data', 'docs', 'environment.yml', 'models', 'options', 'scripts', 'test.py', 'train.py', 'util', 'cache']


In [3]:
#Importing a bunch more libraries
import numpy as np
import matplotlib.pyplot as plt
import torch  # Pytorch
import torch.nn as nn  # Neural network module
import torch.nn.functional as fn  # Function module
from torchvision import datasets  # Datasets from torchvision
from torchvision import transforms  # Transforms from torchvision
from torch.utils.data import DataLoader
from matplotlib import animation
import sys
import progressbar

In [4]:
!pip install pycm livelossplot
%pylab inline
from livelossplot import PlotLosses

Populating the interactive namespace from numpy and matplotlib


In [5]:
import random 
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True

device = 'cuda'  # Set out device to GPU

print('Cuda installed, running on GPU!')  # print sentence

Cuda installed, running on GPU!


In [6]:
def loadfpc(index):
  vels = open("./csv_data/data_"+ str(index) + ".csv")
  velocities = np.loadtxt(vels, delimiter=",")[:,3:5]
  EdgeFeatures = np.array([[(velocities[edge[0]][0]+velocities[edge[1]][0])/2,(velocities[edge[0]][1]+velocities[edge[1]][1])/2] for edge in fpcmesh.edges])
  velx = torch.Tensor([[EdgeFeatures[:,0],EdgeFeatures[:,1]]])
  return velx

In [7]:
#Opt class from MeshCNN, to create a "Mesh" class you need an option class for data augmentation purposes, hard to get mesh loading to work otherwise
class opt:
  num_aug = 1

In [8]:
#Creating my flow past cylinder mesh
fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
fpcmesh.init_history()

In [9]:
def index_split(train_ratio, valid_ratio, test_ratio, total_num = 300):
    if train_ratio + valid_ratio + test_ratio != 1:
        raise ValueError("Three input ratio should sum to be 1!")
    total_index = np.arange(1, total_num)
    rng = np.random.default_rng()
    total_index = rng.permutation(total_index)
    knot_1 = int(total_num * train_ratio)
    knot_2 = int(total_num * valid_ratio) + knot_1
    train_index, valid_index, test_index = np.split(total_index, [knot_1, knot_2])
    return train_index, valid_index, test_index

In [10]:
def load_tensor(simulation_indexes):
    total = len(simulation_indexes)
    cnt_progress = 0
    bar=progressbar.ProgressBar(maxval=total)
    tensor = loadfpc(0)
    cnt_progress+=1
    bar.update(cnt_progress)    
    for i in range(1, total):
        tensor = torch.cat((tensor, loadfpc(i)))
        cnt_progress+=1
        bar.update(cnt_progress)
    bar.finish()
    return tensor

In [11]:
train_ratio = 0.8
valid_ratio = 0.1
test_ratio = 0.1
train_index, valid_index, test_index = index_split(train_ratio, valid_ratio, test_ratio, total_num = 1000)

In [12]:
# train_set = load_tensor(train_index)
# valid_set = load_tensor(valid_index)
# test_set = load_tensor(test_index)

In [13]:
# torch.save(train_set, './TENS/train_tensor.pt')
# torch.save(valid_set, './TENS/valid_tensor.pt')
# torch.save(test_set, './TENS/test_tensor.pt')

In [14]:
train_set = torch.load('./TENS/train_tensor.pt', map_location=torch.device('cpu'))
valid_set = torch.load('./TENS/valid_tensor.pt', map_location=torch.device('cpu'))
test_set = torch.load('./TENS/test_tensor.pt', map_location=torch.device('cpu'))

In [15]:
encdek = networks.MeshEncoderDecoder([10421,5000,3000,1500,750],[2,2,2,2,2],[2,2,2,2,2],transfer_data = True)

In [16]:
fc1 = nn.Linear(750,256)
fc2 = nn.Linear(256,128)
fc3 = nn.Linear(128,256)
fc4 = nn.Linear(256,700)

In [17]:
FCAE = nn.Sequential(fc1,fc2,fc3,fc4)

In [18]:
encdek2 = nn.Sequential(encdek.encoder,FCAE, encdek.decoder)

In [19]:
encdek3 = networks.MeshEncoderDecoder([10421,5000],[2,2,2],[2,2,2],transfer_data = True)

In [23]:
train_set[20]

tensor([[3.9074e-02, 3.9036e-02, 3.9053e-02,  ..., 4.0041e-02, 3.9728e-02,
         3.9734e-02],
        [3.2951e-05, 2.8814e-05, 2.0752e-05,  ..., 4.9996e-06, 4.2264e-05,
         1.0473e-04]])

In [22]:
x = torch.Tensor()

ValueError: ignored

In [20]:
result = encdek3.forward([train_set[20]],[fpcmesh])

AttributeError: ignored

In [ ]:
train_set[20].size()

In [ ]:
def train(autoencoder, optimizer, criterion, dataloader):
  autoencoder.train()
  train_loss, data_length = 0, len(dataloader.dataset)
  for batch in dataloader:
      meshes = []
      for i in range(dataloader.batch_size):
        fpcmesh = mesh.Mesh("fpc3.obj",opt=opt)
        fpcmesh.init_history()
        meshes.append(fpcmesh)
      batch = batch.to(device)  # Send batch of images to the GPU
      optimizer.zero_grad()  # Set optimiser grad to 0
      x_hat = autoencoder(batch,meshes)  # Generate predicted images (x_hat) by running batch of images through autoencoder
      # print("hello")
      # print(batch.size(),x_hat.size())
      MSE = criterion(batch, x_hat)  # Calculate MSE loss
      print(MSE)
      MSE.backward()  # Back-propagate
      optimizer.step()  # Step the optimiser
      train_loss += MSE * batch.size(0)

  return train_loss / data_length  # Return MSE

def validate(autoencoder, optimizer, criterion, dataloader):
    autoencoder.eval()
    validation_loss, data_length = 0, len(dataloader.dataset)
    for batch in dataloader:
        with torch.no_grad():
            batch = batch.to(device)  # Send batch of images to the GPU
            x_hat = autoencoder(batch)  # Generate predicted images (x_hat) by running batch of images through autoencoder
            MSE = criterion(batch, x_hat)  # Calculate MSE loss
            validation_loss += MSE * batch.size(0)

    return validation_loss / data_length   # Return MSE

In [ ]:
def train_model(autoencoder, batch_size=4, n_epochs = 10, weight_decay = 1e-5, visualize=True):
  set_seed(42)
  autoencoder = autoencoder.to(device)
  optimizer = torch.optim.Adam(autoencoder.parameters())

  # we choose the MSE to be the loss function 
  criterion = nn.MSELoss()

  train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
  valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)
  
  # do livelossplot if visualize turned-on
  if visualize:
      liveloss = PlotLosses()

  for epoch in range(n_epochs):
    train_MSE = train(autoencoder, optimizer, criterion, train_loader)
    validation_MSE = validate(autoencoder, optimizer, criterion, valid_loader)
    print("eppoch %d starting......"%(epoch))
    
    # do livelossplot if visualize turned-on 
    if visualize: 
      logs = {}

      logs['' + 'log loss'] = train_MSE.item()
      logs['val_' + 'log loss'] = validation_MSE.item()

      liveloss.update(logs)
      liveloss.draw()

      
  return autoencoder

In [ ]:
autoencoder = train_model(encdek)